In [20]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import json

In [ ]:
#load data
with open('diet-data.json', 'r' ) as file:
    data = json.load(file)


In [95]:
# assign for problem values 
foods = data['foods']
nutrion_sample = data['nutritionValues']
nutritionValues = {}
min_nutrion = data['minNutrition']
max_nutrion = data['maxNutrition']
categories =data['categories']

# converting nutrionvalues list to dictionary type
for i in range(0,len(nutrion_sample)):

    nutritionValues.update({tuple(nutrion_sample[i][0]): nutrion_sample[i][1]})


In [104]:

model = gp.Model('diet_estimate')
buy = model.addVars(foods,name = 'buy')

# assign for minimun of cost values
model.setObjective(buy.prod(data['cost']), GRB.MINIMIZE)

# assign for minimum values of nutrions 
model.addConstrs(
    (
        gp.quicksum(nutritionValues[f, c] * buy[f] for f in foods)
        == [min_nutrion[c], max_nutrion[c]]
        for c in categories
    ),
    "_",
)
# assign for non-negative food purchased 
model.addConstrs((buy[f] >=0 for f in foods), name='c1')


{'hamburger': <gurobi.Constr *Awaiting Model Update*>,
 'chicken': <gurobi.Constr *Awaiting Model Update*>,
 'hot dog': <gurobi.Constr *Awaiting Model Update*>,
 'fries': <gurobi.Constr *Awaiting Model Update*>,
 'macaroni': <gurobi.Constr *Awaiting Model Update*>,
 'pizza': <gurobi.Constr *Awaiting Model Update*>,
 'salad': <gurobi.Constr *Awaiting Model Update*>,
 'milk': <gurobi.Constr *Awaiting Model Update*>,
 'ice cream': <gurobi.Constr *Awaiting Model Update*>}

In [111]:
model.optimize()
if model.Status == GRB.OPTIMAL:
    for f in foods:
        if buy[f].X > 1e-6:   # only show foods that are actually chosen
            print(f"{f}: {buy[f].X}")

    print("\nTotal nutrient consumption:")
    for c in categories:
        value = sum(nutritionValues[f, c] * buy[f].X for f in foods)
        if c == 'sodium':
            value_sodium = value
        print(f"{c}: {value:.2f} (min {min_nutrion[c]}, max {max_nutrion[c]})")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 13 rows, 12 columns and 48 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.182886111e+01
hamburger: 0.6045138888888888
milk: 6.970138888888889
ice cream: 2.5913194444444447

Total nutrient consumption:
calories: 1800.00 (min 1800, max 2200)
protein: 91.00 (min 91, max 1e+100)
fat: 59.06 (min 0, max 65)
sodium: 1779.00 (min 0, max 1601.1000000000001)


In [114]:
max_nutrion.update({'sodium': value_sodium * 0.9})

model2 = gp.Model('diet_estimate2')
buy2 = model2.addVars(foods,name = 'buy2')

# assign for minimun of cost values
model2.setObjective(buy2.prod(data['cost']), GRB.MINIMIZE)

# assign for minimum values of nutrions 
model2.addConstrs(
    (
        gp.quicksum(nutritionValues[f, c] * buy2[f] for f in foods)
        == [min_nutrion[c], max_nutrion[c]]
        for c in categories
    ),
    "_",
)
# assign for non-negative food purchased 
model2.addConstrs((buy2[f] >=0 for f in foods), name='c1')

{'hamburger': <gurobi.Constr *Awaiting Model Update*>,
 'chicken': <gurobi.Constr *Awaiting Model Update*>,
 'hot dog': <gurobi.Constr *Awaiting Model Update*>,
 'fries': <gurobi.Constr *Awaiting Model Update*>,
 'macaroni': <gurobi.Constr *Awaiting Model Update*>,
 'pizza': <gurobi.Constr *Awaiting Model Update*>,
 'salad': <gurobi.Constr *Awaiting Model Update*>,
 'milk': <gurobi.Constr *Awaiting Model Update*>,
 'ice cream': <gurobi.Constr *Awaiting Model Update*>}

In [113]:
model2.optimize()
if model2.Status == GRB.OPTIMAL:
    for f in foods:
        if buy2[f].X > 1e-6:   # only show foods that are actually chosen
            print(f"{f}: {buy2[f].X}")

    print("\nTotal nutrient consumption:")
    for c in categories:
        value = sum(nutritionValues[f, c] * buy2[f].X for f in foods)
        print(f"{c}: {value:.2f} (min {min_nutrion[c]}, max {max_nutrion[c]})")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 13 rows, 12 columns and 48 nonzeros
Model fingerprint: 0x9d14fbf5
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]
Presolve removed 9 rows and 2 columns
Presolve time: 0.02s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.472500e+02   0.000000e+00      0s
       4    1.2107477e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.10 seconds (0.00 work units)
Optimal objective  1.210747698e+01
hamburger: 0.06328373015873043
milk: 8.334980158730158
ice cream: 2.8501686507936506

Total nutrient consumption:
calories: 1800.00 (min 1800, max 